# Введение в машинное обучение для Java-разработчиков
### Практическое задание 2.  Деревья решений, случайный лес.
### Дата выдачи: 26.10.2023

### Дедлайн: 23:59MSK 08.11.2023

## О задании
В этом задании мы попытаемся разобраться в устройстве деревьев решений и ансамбля на основе деревьев. 

## Оценивание и штрафы
Каждая из задач (помечены тегом [task]) имеет определенное количество баллов (указана в скобках около задачи). Максимально допустимая оценка за работу — 15 баллов. 

- от 4 до 8 баллов - оценка "3"
- от 8 до 14 баллов - оценка "4"
- 15 баллов - оценка "5"

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов, что автоматически ведет к несдаче курса. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в комментариях. 
В данном задании есть необязательные бонусные задания, выполнение которых добавляет баллы в карму :)

## Формат сдачи
Задания сдаются путем форка основного репозитория, коммита решения в мастер-ветку вашего форка и оповещении преподавателя о выполнении ДЗ. 

Для удобства проверки самостоятельно посчитайте свою максимальную оценку (исходя из набора решенных задач) и укажите ниже.

[[Укажите количество набранных баллов]]

## Часть 1. Дерево решений

- [task] Реализуйте подбор признака и порога для поиска условия разбиения выборки методом перебора (2 балла)
- [task] Реализуйте метод predict для инференса дерева на датасете (1 балл)
- [task] Обучите дерево на датасете ирисов Фишера (1 балл)
- Бонусное задание: В методе `_calculate_leaf_value` возвращайте вероятность классов
- Бонусное задание: Замените критерий Джини на энтропийный критерий
4 балла

In [142]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

class DecisionNode:
    def __init__(self, feature=None, threshold=None, value=None, true_branch=None, false_branch=None):
        self.feature = feature
        self.threshold = threshold
        self.value = value
        self.true_branch = true_branch
        self.false_branch = false_branch

    def predict_one(self, x):
        if self.feature is not None:
            if x[self.feature] <= self.threshold:
                return self.true_branch.predict_one(np.array(x))
            else:
                return self.false_branch.predict_one(np.array(x))
        else:
            return self.value

    

class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=5):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, y):
        self.root = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        if depth == self.max_depth or len(y) < self.min_samples_split:
            return DecisionNode(value=self._calculate_leaf_value(y))

        best_feature, best_threshold = self._find_best_split(X, y)
        if best_feature is None or best_threshold is None:
            return DecisionNode(value=self._calculate_leaf_value(y))

        mask = X[:, best_feature] <= best_threshold
        true_branch = self._build_tree(X[mask], y[mask], depth + 1)
        false_branch = self._build_tree(X[~mask], y[~mask], depth + 1)

        return DecisionNode(feature=best_feature, threshold=best_threshold, true_branch=true_branch, false_branch=false_branch)

    def _find_best_split(self, X, y):
        best_gain = 0
        best_feature = None
        best_threshold = None
        n_features = X.shape[1]
        X_t = np.transpose(X)
        
        for i in range(n_features):
            for threshold in np.unique(X_t[i]):
                y_true = [res for j, res in enumerate(y) if X[j][i] > threshold]
                y_false = [res for j, res in enumerate(y) if X[j][i] <= threshold]               
                gini = self._calculate_gain(y, y_true, y_false)
                if gini > best_gain:
                    best_gain = gini
                    best_feature = i
                    best_threshold = threshold
        return best_feature, best_threshold

    def _calculate_gain(self, y, y_true, y_false):
        # print(y)
        # print(y_true)
        # print(y_false)

        p = len(y_true) / len(y)
        impurity_before = self._entropy(y)
        # print(f"{impurity_before=}")
        impurity_after = p * self._entropy(y_true) + (1 - p) * self._entropy(y_false)
        # print(f"{impurity_after=}")
        return impurity_before - impurity_after

    def _gini_impurity(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities ** 2)

    def _entropy(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum(probabilities*np.log(probabilities))

    def _calculate_leaf_value(self, y):
        classes, counts = np.unique(y, return_counts=True)
        ans = np.zeros(max(classes) + 1)
        for clas, count in zip(classes, counts):
            ans[clas] = count / len(y)
        return ans

    def predict(self, X):
        ans = []
        for i, x in enumerate(X):
            ans.append(self.root.predict_one(x))
        return ans
            

def print_tree(node, depth=0):
    indent = "    " * depth
    if node.value is not None:
        print(indent + "Predicted value:", node.value)
    else:
        print(indent + "Feature", node.feature, "<=", node.threshold)
        print(indent + "--> True:")
        print_tree(node.true_branch, depth + 1)
        print(indent + "--> False:")
        print_tree(node.false_branch, depth + 1)

In [143]:
data = load_iris(as_frame=True)
X = data["data"].values
y = data["target"].values
tree = DecisionTree(max_depth=2) # на 5 он переобучается и выдает всегда правильно с вероятностью в единицу, так жить неинтересно
tree.fit(X, y)

y_predicted = tree.predict(X)

for y_orig, y_pred in zip(y, y_predicted):
    print(f"y={y_orig}, predicted probabilities = {dict(enumerate(y_pred))}")


y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}
y=0, predicted probabilities = {0: 1.0}


### Часть 2. Случайный лес

##### [task] 1. Усреднение классификаторов (2 балла)
Реализуйте бэггинг над решающими деревьями (усреднение предсказанных вероятностей всего ансамбля или путем "голосования"). В качестве базового алгоритма используйте либо наш класс или DecisionTreeClassifier из пакета scikit-learn. 

##### [task] 2. Бутстрап обучающей выборки (2 балла)
Добавим к нашему усреднению предсказаний бутстрап выборки (генерация случайной выборки того же размера с возвращением). Сгенерируйте с помощью него отдельную обучающую выборку для каждого дерева, обучите их и усредните предсказания, как в предыдущем пункте.

##### [task] 3. Выбор случайного подмножества признаков при построении нового дерева. (2 балла)
Обучайте каждое дерево на отдельной бутстрап-выборке и случайно выбирайте признаки для обучения. 

##### [task] 4. Подсчитайте метрики оценки качества классификации (accuracy, precision, recall, ROC AUC, F-мера) для каждого из вариантов дерева. Сделайте отдельную функцию для подсчета метрик (1 балл)

- Используемый датасет https://archive.ics.uci.edu/dataset/94/spambase
- Все гиперпараметры необходимо вынести аргументы соответсвующей функции.

In [238]:
# fetch dataset 
!pip install ucimlrepo

from ucimlrepo import fetch_ucirepo 
spambase = fetch_ucirepo(id=94) 
  
# data (as numpy arrays) 
X = spambase.data.features.values
y = spambase.data.targets.values
  
# metadata 
print(spambase.metadata) 
  
# variable information 
print(spambase.variables)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

{'uci_id': 94, 'name': 'Spambase', 'repository_url': 'https://archive.ics.uci.edu/dataset/94/spambase', 'data_url': 'https://archive.ics.uci.edu/static/public/94/data.csv', 'abstract': 'Classifying Email as Spam or Non-Spam', 'area': 'Computer Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 4601, 'num_features': 57, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['Class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1999, 'last_updated': 'Mon Aug 28 2023', 'dataset_doi': '10.24432/C53G6X', 'creators': ['Mark Hopkins', 'Erik Reeber', 'George Forman', 'Jaap Suermondt'], 'intro_paper': None, 'additional_info': {'summary': 'The "spam" concept is diverse: advertisements for products/web sites, make money fast schemes, chain letters, pornography...\n\nThe classification task for this dataset is to determine whether a given email is spam or not.\n\t\nOur collecti

In [276]:
import sklearn.model_selection 
import random
num_of_trees = 5

def generate_masks(num_of_inps, num_of_masks):
    ans = np.zeros([num_of_masks, num_of_inps], dtype=bool)
    took_inps = np.array([False]*num_of_inps, dtype=bool)
    for i in range(num_of_masks):
        ans[i] = np.random.random(num_of_inps) > 0.5
        took_inps = took_inps | ans[i]
    if not took_inps.all():
        ans[i] |= ~took_inps
        
    return ans

def bootstrap(inp):
    ans = np.zeros(inp.shape)
    for i in range(inp.shape[0]):
        ans[i] = random.choice(inp)
    return ans

masks = generate_masks(X.shape[1], num_of_trees)

conc = np.concatenate((X, y), axis=1)
datasets_conc = [bootstrap(conc) for _ in range(num_of_trees)]
datasets_X = [dataset_conc[:, :-1] for dataset_conc in datasets_conc]
datasets_Y = [dataset_conc[:, -1] for dataset_conc in datasets_conc]


from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier


forest = [DecisionTreeClassifier() for _ in range(num_of_trees)]
for i, tree in enumerate(forest):
    tree.fit(datasets_X[i][:, masks[i]], datasets_Y[i])


def predict(X, forest, masks):
    result = None
    for i, tree in enumerate(forest):
        if result is None:
            result = tree.predict(X[:, masks[i]])
        else:
            result += tree.predict(X[:, masks[i]])
    return result / len(forest)

pred = predict(X, forest, masks)

print(pred)
print(y.T[0])

print(np.sum(np.abs(pred- y.T[0])) / len(y))




[1. 1. 1. ... 0. 0. 0.]
[1 1 1 ... 0 0 0]
0.11371440991088892


#### Часть 3. 

- [task] Обучите RandomForestClassifier из sklearn на датасете из прошлой части (2 балла)
- [task] Подсчитайте accuracy, precision, recall, ROC AUC, F-мера на отложенной выборке. Получилось лучше или хуже по сравнению с вашим вариантом RandomForest (2 балла)
- Попробуйте объяснить результат.

In [283]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

clf = RandomForestClassifier(n_estimators=5, max_depth=5)
clf.fit(X, y.T[0])

def findTruePositives(model, X, y):
    return np.rint(model.predict(X)[y == 1]).sum()

def findTrueNegatives(model, X, y):
    return (np.rint(model.predict(X)[y == 0]) == 0).sum()

def findFalsePositives(model, X, y):
    return np.rint(model.predict(X)[y == 0]).sum()

def findFalseNegatives(model, X, y):
    return (np.rint(model.predict(X)[y == 1]) == 0).sum()
    
TP = findTruePositives(clf, X, y.T[0])
FP = findFalsePositives(clf, X, y.T[0])
TN = findTrueNegatives(clf, X, y.T[0])
FN = findFalseNegatives(clf, X, y.T[0])
accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
F_score = 2 * (precision * recall) / (precision + recall)
roc = roc_auc_score(y.T[0], clf.predict(X))

print(f"{TP=} {FP=} {TN=} {FN=}")
print(f"{accuracy=}")
print(f"{precision=}")
print(f"{recall=}")
print(f"{F_score=}")
print(f"{roc=}")

TP=1542.0 FP=146.0 TN=2642 FN=271
accuracy=0.9093675287980874
precision=0.9135071090047393
recall=0.8505239933811363
F_score=0.8808911739502999
roc=0.8990783525011852


In [285]:
class Stub:
    pass

ownModel = Stub()
ownModel.predict = lambda X: predict(X, forest, masks)
    
TP = findTruePositives(ownModel, X, y.T[0])
FP = findFalsePositives(ownModel, X, y.T[0])
TN = findTrueNegatives(ownModel, X, y.T[0])
FN = findFalseNegatives(ownModel, X, y.T[0])

accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
F_score = 2 * (precision * recall) / (precision + recall)
roc = roc_auc_score(y.T[0], ownModel.predict(X))

print(f"{TP=} {FP=} {TN=} {FN=}")
print(f"{accuracy=}")
print(f"{precision=}")
print(f"{recall=}")
print(f"{F_score=}")
print(f"{roc=}")

TP=1537.0 FP=62.0 TN=2726 FN=276
accuracy=0.9265377091936535
precision=0.9612257661038149
recall=0.8477661334804192
F_score=0.9009378663540446
roc=0.9619789247274387


На таких близких числах сложно о чем-то говорить, тем более что опытным путем выяснилось, что если модельки "поддушить" (снизить максимальную глубину + ограничить колво деревьев),
то собственная модель почему-то начинает показывать себя лучше (хотя обе они построены на DecisionTreeClassifier), заметное отличие только в ROC, что вероятно связано с тем, 
что собственная модель все-таки выдает среднее значение по всем деревьям, а sklearn'вская проводит голосование